In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"", regex=True)

# Check names.
student_data_df.head(10)

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [ ]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [ ]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") & (student_data_df["grade"] == "9th"),"reading_score"]=np.nan


In [ ]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") & (student_data_df["grade"] == "9th"),"math_score"] = np.nan

In [ ]:
#  Step 4. Check the student data for NaN's. 
student_data_df.tail()

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [ ]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

In [ ]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [ ]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [ ]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
thomas_nine_count = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & (school_data_complete_df["grade"] == "9th")].count()
# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()

# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
student_count_new = student_count - thomas_nine_count

In [ ]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

#Convert math and reading scores to float data type

In [ ]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = passing_math_count/(student_count_new) * 100
passing_reading_percentage = passing_reading_count/(student_count_new) * 100

In [ ]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()

# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = overall_passing_math_reading_count / student_count_new *100

In [ ]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])


# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

##  School Summary

In [ ]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [ ]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


# per_school_summary_df.head()

In [ ]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df

In [ ]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).


In [ ]:
# Step 6. Get all the students passing math from THS


In [ ]:
# Step 7. Get all the students passing reading from THS


In [ ]:
# Step 8. Get all the students passing math and reading from THS


In [ ]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 


In [ ]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.


In [ ]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 



In [ ]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.


In [ ]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.


In [ ]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.


In [ ]:
# per_school_summary_df

## High and Low Performing Schools 

In [ ]:
# Sort and show top five schools.


In [ ]:
# Sort and show top five schools.


## Math and Reading Scores by Grade

In [ ]:
# Create a Series of scores by grade levels using conditionals.


# Group each school Series by the school name for the average math score.


# Group each school Series by the school name for the average reading score.


In [ ]:
# Combine each Series for average math scores by school into single data frame.


In [ ]:
# Combine each Series for average reading scores by school into single data frame.


In [ ]:
# Format each grade column.


In [ ]:
# Remove the index.


# Display the data frame


In [ ]:
## Remove the index.


# Display the data frame


## Scores by School Spending

In [ ]:
# Establish the spending bins and group names.


# Categorize spending based on the bins.


In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Create the DataFrame


In [ ]:
# Format the DataFrame 


## Scores by School Size

In [ ]:
# Establish the bins.

# Categorize spending based on the bins.


In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Assemble into DataFrame. 


In [ ]:
# Format the DataFrame  


## Scores by School Type

In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Assemble into DataFrame. 


In [ ]:
# # Format the DataFrame 


In [ ]:
# Add pandas dependency
import pandas as pd

In [ ]:
# Create variables for files to load, with path to files"
school_data_to_load= "Resources/schools_complete.csv"
student_data_to_load= "Resources/students_complete.csv"

In [ ]:
# Read the school data file and store it in a Pandas DataFrame.
school_data_df = pd.read_csv(school_data_to_load)
school_data_df.head()

In [ ]:
# Read the student data file and store it in a Pandas DataFrame.
student_data_df = pd.read_csv(student_data_to_load)
student_data_df.head()


In [ ]:
# Determine if there are missing values in school data
school_data_df.count()

In [ ]:
# Determine if there are missing values in student data
student_data_df.count()


In [ ]:
school_data_df.dtypes

In [ ]:
student_data_df.dtypes

In [ ]:
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"",regex=True)
student_data_df

In [ ]:
# Combine the data into a single dataset. New data frame is called school_data_complete_df
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])
school_data_complete_df.head()

In [ ]:
# Check count of values to make sure everything is consistent
student_count = school_data_complete_df['Student ID'].count()
student_count

In [ ]:
# Get count of students
school_data_complete_df['Student ID'].count()

In [ ]:
# Calculate the total number of schools, using count method from original school dataframe
school_count = school_data_df["school_name"].count()
school_count

In [ ]:
# Calculate the total number of schools, using new merged data frame and "unique" method. This outputs an array
school_count_2 = school_data_complete_df["school_name"].unique()
school_count_2

In [ ]:
# Use len() to determine length of the array, and to determine number of unique schools
len(school_data_complete_df["school_name"].unique())

In [ ]:
# Calculate the total budget. Use school data dataframe, not merged data frame to make sure we are adding each school's budget only once
total_budget = school_data_df["budget"].sum()
total_budget

In [ ]:
# Calculate the average reading score
average_reading_score = school_data_complete_df["reading_score"].mean()
average_reading_score

In [ ]:
# Calculate the average math score
average_math_score = school_data_complete_df["math_score"].mean()
average_math_score

In [ ]:
# Assign variable and determine passing grades for math. True= passing, false= not passing
passing_math = school_data_complete_df["math_score"] >= 70
passing_math.head()

In [ ]:
# Assign variable and determine passing grades for reading. True= passing, false= not passing
passing_reading = school_data_complete_df["reading_score"] >= 70
passing_reading.head()

In [ ]:
# Get all the students who are passing math in a new DataFrame.
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_math.head()

In [ ]:
# Get all the students who are passing reading in a new DataFrame.
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_reading.head()

In [ ]:
# Get count of students passing math
passing_math_count = passing_math['student_name'].count()
# Get count of students passing reading
passing_reading_count = passing_reading['student_name'].count()
print (passing_math_count)
print (passing_reading_count)

In [ ]:
# Calculate percentages of students who passed math and who passed reading
passing_math_percentage = passing_math_count/float(student_count) * 100
passing_reading_percentage = passing_reading_count/float(student_count) * 100
print(passing_math_percentage)
print(passing_reading_percentage)

In [ ]:
# Calculate the students who passed both math and reading, using & and above code
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]

passing_math_reading.head()

In [ ]:
# Calculate the number of students who passed both math and reading.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
overall_passing_math_reading_count

In [ ]:
# Calculate the overall passing percentage.
overall_passing_percentage = overall_passing_math_reading_count / student_count * 100
overall_passing_percentage

In [ ]:
# Adding a list of values with keys to create a new DataFrame. Keys are in quotes, values are variables we have created above.
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count,
          "Total Students": student_count,
          "Total Budget": total_budget,
          "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])
district_summary_df

In [ ]:
# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

district_summary_df["Total Students"]

In [ ]:
# Format "Total Budget" to have the comma for a thousands separator, a decimal separator, and a "$".

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df["Total Budget"]

In [ ]:
# Define a function that calculates the percentage of students that passed both # math and reading and prints the passing percentage to the output when the
# function is called.
def passing_math_percent(pass_math_count, student_count):
    return pass_math_count / float(student_count) * 100

In [ ]:
passing_math_count = 29370
total_student_count = 39170
# Call the function.
passing_math_percent(passing_math_count, total_student_count)

In [ ]:
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)

district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)

district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)

In [ ]:
# Display the summary dataframe with new formatted lines.
district_summary_df

In [ ]:
# Determine the school type
per_school_types = school_data_df.set_index(["school_name"])["type"]
per_school_types

In [ ]:
# Add per_school_types as a Dataframe for testing
df = pd.DataFrame(per_school_types)
df

In [ ]:
# Get the per school counts using the schools dataframe and "size" column
per_school_counts = school_data_df["size"]
per_school_counts

In [ ]:
# Get the per school counts using school dataframe and "size" column, setting school_name as index
per_school_counts = school_data_df.set_index(["school_name"])["size"]
per_school_counts

In [ ]:
# # Get the per school counts using school_complete dataframe and the value_counts method on the "school_name" column
# per_school_counts = school_data_complete_df['school_name'].value_counts()
# per_school_counts

In [ ]:
# Get per school budget using school_data dataframe and 'budget' column, setting index as 'school_name'
per_school_budget = school_data_df.set_index(['school_name'])['budget']
per_school_budget

In [ ]:
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts
per_school_capita

In [ ]:
# Calculate the average math scores. The .groupby() function does an operation on each column, in this case .mean()
per_school_averages = school_data_complete_df.groupby(["school_name"]).mean()
per_school_averages

In [ ]:
# Calculate the average test scores, using .groupby and mean. With 'school_name' as the index and only looking at either "math score" or "reading score"
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]
per_school_math

In [ ]:
# Output reading score
per_school_reading

In [ ]:
# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

In [ ]:
per_school_passing_math.head()

In [ ]:
# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

In [ ]:
# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

In [ ]:
# Calculate the students who passed both math and reading.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading.head()

In [ ]:
# Calculate the number of students who passed both math and reading.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

In [ ]:
# Calculate the overall passing percentage.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100
per_overall_passing_percentage

In [ ]:
# Adding a list of values with keys to create a new DataFrame.

per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_overall_passing_percentage})
per_school_summary_df.head()

In [ ]:
# Format the Total School Budget and the Per Student Budget columns.
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)

per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)


# Display the data frame
per_school_summary_df.head()

In [ ]:
# Sort and show top five schools. sort_values on a given column, ascending=false to specify highest to lowest 
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_schools.head()

In [ ]:
# Sort and show top five schools. sort_values on the "% overall passing" column, ascending= true is default (lowest to highest)
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head()

In [ ]:
# Create a grade level DataFrames. Referencing school complete dataframe, setting 'grade' equal to 9th,10th,etc
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

In [ ]:
#  Group each grade level DataFrame by the school name for the average math score. Use groupby school_name as index, taking mean of math_scores
ninth_grade_math_scores = ninth_graders.groupby(['school_name']).mean()['math_score']
tenth_grade_math_scores = tenth_graders.groupby(['school_name']).mean()['math_score']
eleventh_grade_math_scores = eleventh_graders.groupby(['school_name']).mean()['math_score']
twelfth_grade_math_scores = twelfth_graders.groupby(['school_name']).mean()['math_score']
eleventh_grade_math_scores

In [ ]:
#  Group each grade level DataFrame by the school name for the average reading score. Use groupby school_name as index, taking mean of reading_scores
ninth_grade_reading_scores = ninth_graders.groupby(['school_name']).mean()['reading_score']
tenth_grade_reading_scores = tenth_graders.groupby(['school_name']).mean()['reading_score']
eleventh_grade_reading_scores = eleventh_graders.groupby(['school_name']).mean()['reading_score']
twelfth_grade_reading_scores = twelfth_graders.groupby(['school_name']).mean()['reading_score']
twelfth_grade_reading_scores

In [ ]:
# Combine each grade level Series for average math scores by school into a single dataframe
math_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_math_scores,
    '10th': tenth_grade_math_scores,
    '11th': eleventh_grade_math_scores,
    '12th': twelfth_grade_math_scores})
math_scores_by_grade.head()

In [ ]:
# Combine each grade level Series for average reading scores by school into a single dataframe
reading_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_reading_scores,
    '10th': tenth_grade_reading_scores,
    '11th': eleventh_grade_reading_scores,
    '12th': twelfth_grade_reading_scores})
reading_scores_by_grade.head()

In [ ]:
# Format each column using the .map() function
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

# Remove the Index name
math_scores_by_grade.index.name = None
# Display the dataframe
math_scores_by_grade.head()

In [ ]:
# Format each column using the .map() function
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:.1f}".format)

# Remove the Index name
reading_scores_by_grade.index.name = None
# Display the dataframe
reading_scores_by_grade.head()

In [ ]:
# Get descritpive statistics for per_school_capita which is budget/student for each school
per_school_capita.describe()

In [ ]:
# Assign spending bins 
spending_bins = [0,585,615,645,675]
# Cut per_school_capita data using spending ranges
pd.cut(per_school_capita,spending_bins)
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

In [ ]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [ ]:
# Categorize spending based on bins
per_school_summary_df['Spending Ranges (Per Student)'] = pd.cut(per_school_capita, spending_bins, labels=group_names)
per_school_summary_df

In [ ]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
overall_passing_spending

In [ ]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

spending_summary_df

In [ ]:
# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df

In [ ]:
# Establish Bins based on school size
size_bins = [0,1000,2000,5000]
group_names = ["Small <1000", "Medium 1000-2000", "Large 2000-5000"]

In [ ]:
# Categorize spending based on bins
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels= group_names)
per_school_summary_df.head()

In [ ]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]

size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
size_overall_passing

In [ ]:
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
size_summary_df

In [ ]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)
size_summary_df

In [ ]:
# Calculate averages for the desired columns. Getting average scores and passing percentage by school type
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]

type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]


In [ ]:
type_overall_passing

In [ ]:
# Assemble into DataFrame.
type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

type_summary_df


In [ ]:
# Format dataframe
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df